In [225]:
import requests
import urllib.request
import urllib.parse
import json
from bs4 import BeautifulSoup
from pymongo import MongoClient
import re

#DB connexion
global client 
client = MongoClient(port=27017)
global db
db=client.GeniusScrappingProject2

#genius API base and client_token
base = "https://api.genius.com"
client_access_token = "z1va0QLLfH2tm_2NErEGiCUvLUJS0RMBMzfx17EEzUnj3wvJOu4_Dvjx6CNJyJSW"

In [226]:
def get_existing_songs():
    songs = db.songs.find()
    existing_songs = []
    for song in songs:
        existing_songs.append([song['id'], song['primary_artists'].items()])
    return existing_songs

In [227]:
def get_primary_artists_from_songs():
    songs = db.songs.find()
    existing_songs = {}
    for i, song in enumerate(songs):
        existing_songs.update({
            song['primmary_artists']
        })
    return existing_songs

In [228]:
#returns list of IDs of the artists we have in the DB
def get_list_artists():
    artists = db.artists.find()
    existing_artists = []
    for artist in artists:
        existing_artists.append(artist['id'])
    return existing_artists

In [229]:
def get_artists_without_songs_in_db():
    all_artists = get_list_artists()
    artists_with_songs = get_artists_with_songs_in_db()
    artists_without_songs = []
    for artist in all_artists:
        if not artist in artists_with_songs:
            artists_without_songs.append(artist)
    return artists_without_songs

In [230]:
def get_artists_with_songs_in_db():
    existing_songs = get_existing_songs()
    artists_with_songs=[]
    list_atists = get_list_artists()
    for song in existing_songs:
        if sorted(song[1])[0][1] in list_atists:
            artists_with_songs.append(sorted(song[1])[0][1])
    artists_with_songs = list(set(artists_with_songs))
    return artists_with_songs

In [231]:
#add new artists to our DB
def add_artists(artists):
    for artist in artists:
        entry = {
            'id' : int(artists[artist]['id']),
            'name' : artists[artist]['name'],
            'url' : artists[artist]['url'],
        }
        #Step 3: Insert business object directly into MongoDB via isnert_one
        result=db.artists.insert_one(entry)
        print('artists {} added with success'.format(artists[artist]['name']))

In [232]:
#to get the response in json format ***
def get_json(path, params=None, headers=None):

    # Generate request URL
    requrl = '/'.join([base, path])
    token = "Bearer {}".format(client_access_token)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    # Get response object from querying genius api
    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()
    return response.json()

In [233]:
def search(artist_name):
    '''Search Genius API via artist name.'''
    search = "/search?q="
    query = base + search + urllib.parse.quote(artist_name)
    print(query)
    request = urllib.request.Request(query)

    request.add_header("Authorization", "Bearer " + client_access_token)
    request.add_header("User-Agent", "")
    
    response = urllib.request.urlopen(request, timeout=3)
    raw = response.read()
    data = json.loads(raw)['response']['hits']

    for item in data:
        # Print the artist and title of each result
        print(item['result']['primary_artist']['name']
              + ': ' + item['result']['title'])

In [234]:
def search_artist(artist_id):
    '''Search meta data about artist Genius API via Artist ID.'''
    search = "artists/"
    path = search + str(artist_id)
    request = get_json(path)
    data = request['response']['artist']

    print(data["followers_count"])
    # Lots of information we can scrape regarding the artist, check keys
    return data["followers_count"] # number of followers

In [235]:
def connect_lyrics(song_id):
    '''Constructs the path of song lyrics.'''
    url = "songs/{}".format(song_id)
    data = get_json(url)

    # Gets the path of song lyrics
    path = data['response']['song']['path']

    return path

In [236]:
def retrieve_lyrics(song_id):
    '''Retrieves lyrics from html page.'''
    path = connect_lyrics(song_id)

    URL = "http://genius.com" + path
    page = requests.get(URL)

    # Extract the page's HTML as a string
    html = BeautifulSoup(page.text, "html.parser")

    # Scrape the song lyrics from the HTML
    lyrics = html.find("div", class_="lyrics").get_text()
    return lyrics

In [242]:
def get_song_id(artist_id):
    '''Get all the song id from an artist.'''
    current_page = 1
    next_page = True
    songs = [] # to store final song ids

    while next_page:
        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page} # the current page
        data = get_json(path=path, params=params) # get json of songs
        page_songs = data['response']['songs']
        if page_songs:
            # Add all the songs of current page
            songs += page_songs
            # Increment current_page value for next loop
            current_page += 1
            print("Page {} finished scraping".format(current_page))
            # If you don't wanna wait too long to scrape, un-comment this
            if current_page == 2:
                break
        else:
            # If page_songs is empty, quit
            next_page = False

    print("Song id were scraped from {} pages".format(current_page))

    # Get all the song ids, excluding not-primary-artist songs.
    songs = [song["id"] for song in songs
            if song["primary_artist"]["id"] == artist_id]

    return songs

In [238]:
#get the informations of a song
def get_song_information(song_ids):
    
    song_list = {}
    print("Scraping song information")
    for i, song_id in enumerate(song_ids):
        print("id:" + str(song_id) + " start. ->")

        path = "songs/{}".format(song_id)
        data = get_json(path=path)["response"]["song"]
        
        song_list.update({
        i: {
            "id" : data['id'],
            "url": data["url"],
            "title": data["title"],
            "primary_artists" : {re.sub(r"[^\w\s]", '', data["primary_artist"]['name']):data["primary_artist"]['id']},
            "album": data["album"]["name"] if data["album"] else "<single>",
            "release_date": data["release_date"] if data["release_date"] else "unidentified",
            "featured_artists":
                [re.sub(r"[^\w\s]", '', feat["name"]) if data["featured_artists"] else "" for feat in data["featured_artists"]],
            "featured_artists_id":
                [feat['id'] if data["featured_artists"] else "" for feat in data["featured_artists"]],
            "genius_track_id": song_id,
            "genius_album_id": data["album"]["id"] if data["album"] else "none"}
        })

        print("-> id:" + str(data['title']) + " is finished. \n")
    return song_list

In [239]:
#add new songs to the DB
def add_songs(songs):
    existing_songs = get_existing_songs()
    #existingUrls = findExistingUrls()
    entry={}
    for i in songs:
        if not songs[i]['id'] in existing_songs: 
            entry = {
                "id" : songs[i]['id'],
                "url": songs[i]["url"],
                "title": songs[i]["title"],
                "primary_artists" : songs[i]["primary_artists"],
                "album": songs[i]["album"],
                "release_date": songs[i]["release_date"],
                "featured_artists":
                    {feat : songs[i]["featured_artists_id"][j] 
                     if songs[i]["featured_artists"] else "" for j, feat in enumerate(songs[i]["featured_artists"])},
                #"featured_artists_id":
                #    [feat if songs[i]["featured_artists"] else "" for feat in songs[i]["featured_artists_id"]],
                "genius_track_id": songs[i]["genius_track_id"],
                "genius_album_id": songs[i]["genius_album_id"]
            }
            #Step 3: Insert business object directly into MongoDB via isnert_one
            result=db.songs.insert_one(entry)
            #Step 4: Print to the console the ObjectID of the new document
            print('added to DB as {0}'.format(songs[i]['title']))
    #Step 5: Tell us that you are done
    print('finished adding songs')

In [240]:
#def get_producers():


In [245]:
def get_other_artists_from_songs(song_ids):
    '''Retrieve meta data about a song.'''
    # initialize existing artists and a dictionary for new artists.
    existing_artists = get_list_artists()
    artist_list ={}
    print("Scraping song information")
    for i, song_id in enumerate(song_ids):
        print("id:" + str(song_id) + " start. ->")
        path = "songs/{}".format(song_id)
        data = get_json(path=path)["response"]["song"]
        
        if data["featured_artists"]:
            for artist in data["featured_artists"]:
                if not artist['id'] in existing_artists:
                    artist_list.update({
                        int(artist["id"]): {
                            "id": int(artist["id"]),
                            "url": artist["url"],
                            "name": artist["name"]}
                    })
        if data["primary_artist"]:
            if not data["primary_artist"]["id"] in existing_artists:
                artist_list.update({
                    int(data["primary_artist"]["id"]): {
                        "id": int(data["primary_artist"]["id"]),
                        "url": data["primary_artist"]["url"],
                        "name": data["primary_artist"]["name"]}
                })

        print("-> id:" + str(song_id) + " is finished. \n")
        
    return artist_list  

In [246]:
def main():
    # Example searches
    term = 'Eminem'
    artist_list = {}
    artist_without_songs = get_artists_without_songs_in_db()
    artists_with_songs = get_artists_with_songs_in_db()
    artists_ids = get_list_artists()

    existing_songs = get_existing_songs()
    # Grabs all song id's from one artist
    #songs_ids=get_song_id(45)
    
    # Grabs all song id's from artists
    for artist_id in artists_ids:
        if artist_id in artist_without_songs:
            songs_ids=[]
            song_list = []
            songs_ids_of_an_artist = get_song_id(artist_id)
            for song in songs_ids_of_an_artist:
                '''To not scrap songs we already have'''
                if not song in existing_songs:
                    songs_ids.append(song)
            song_list = get_song_information(songs_ids)
            add_songs(song_list)
    
    #Grab all featuring artists from songs ID
    #artist_list = get_other_artists_from_songs(songs_ids)
    
    #add new artists to the DB
    #add_artists(artist_list)
    
    #print(artist_list)
    
    # Get meta information about songs
    #song_list = get_song_information(songs_ids)
    #add_songs(song_list)
    
    # Scrape lyrics from the songs
    # song_lyrics = [retrieve_lyrics(song_id) for song_id in songs_ids]


    # Gets information regarding the artist themself
    # followers = search_artist(artist_id)

    # Shows some random songs from arist and lyrics
    #search(term)


if __name__ == "__main__":
    main()